# Question: Can the business generate growing FCF and sustain high ROIC?

Return on invested capital is the ultimate measure of business profitability and performance. The ROIC metric is the linchpin that connects a company's sales growth, profitability, free cash flow, and balance sheet.

Here's an example from the McKinsey book Valuation. Assume we have two companies (company A and company B) that aim to grow earnings at a rate of 5% per year, but Company A has a ROIC of 20% and Company B has one of only 10%. Then company A only has to reinvest 25% of its profits to grow earnings 5% (20% x 25% = 5%), but company B has to reinvest 50% of its profits to grow earnings at the same 5% rate (10% x 50% = 5%). The company with the higher ROIC has a lower reinvestment rate, and will need to reinvest less capital to achieve the same level of earnings growth. And because the higher-ROIC business requires less capital (reinvestment) to grow, it generates higher free cash flow.

This means ROIC is the prime driver of FCF and earnings growth. ROIC is not only a measure of quality...but also of growth.

Mauboussin writes: "The rate of return on incremental capital is the maximum growth rate in operating profit a business can reach without external financing." Using the formula ROIC x reinvestment rate = profit growth, we can see that a business that generates a ROIC of 10% can't grow operating profit faster than 10%, and to do so it must reinvest 100% of its profits (10% x 100% = 10%). But a company that generates a 20% ROIC and has opportunities to reinvest 100% of its capital can grow earnings at 20% (20% x 100% =20%).

Because these high-ROIC businesses generate so much FCF, they can finance their growth internally, rather than relying on outside capital to grow. This means less debt (or less equity dilution) for shareholders. It also means that some of the excess FCF can be used to pay down debt, which further strengthens the balance sheet. Common terms used to describe high-ROIC businesses that don't require a lot of capital to grow (and therefore generate strong and growing FCF) are "self-funding," "asset-light," and "compounders." No matter what you call them, ESG investors should love them.

ROIC is the North Star in analyzing a company's financial strength and future prospects. Diving into whether a company's ROIC is rising or falling is important, and understanding the drivers behind its ROIC trend line will make you a better investor. The DuPont Analysis breaks ROIC down further to see what's driving returns: high profit margin or high invested-capital turnover.

As part of identifying the drivers, we study a company's business model and its competitive advantages (or its competitive edge), and examine whether these edges are getting sharper or deteriorating. We also study the competitive environment, market-share trends, the rationality (or lack thereof) of pricing in the industry, and barriers to entry. We also employ frameworks like Porter's Five Forces to help with measuring the "moat," or competitive advantage, enjoyed by a company.

Next, we take a look at the net operating profit after tax (NOPAT) -- a crucial number used to calculate FCF, which also serves as the numerator in the ROIC equation; the company's margin profile; and the amount of operating leverage inherent in the business model. We previously demonstrated how to analyze the balance sheet and how efficiently it's managed, because invested capital is the denominator in the ROIC equation.

We also analyze management's ability to allocate capital, the performance metrics used to incentivize management, management's understanding of intrinsic value growth, and the language the company uses when speaking about growing business and shareholder value. Connecting the dots between revenue, ROIC, and FCF is a hugely positive sign.

Lastly, we analyze whether management incorporates ESG into everything it does, because ample research shows that focusing on ESG improves ROIC. Organic revenue growth and high ROIC are ultimately what create the FCF growth (particularly FCF-per-share growth) that we so desire. Remember that FCF is the amount of cash that is available to investors, as it's the amount of surplus cash generated by a business after spending capital (investing) to maintain and grow its assets. This FCF can be returned to shareholders as dividends, share buybacks, or by paying down debt. (If the company paid down all debt, there would be no more creditors to have a claim on cash flow, so equity holders would have full claim -- after the government, which gets its take through taxes. So by paying down debt, management is returning capital to equity holders.) The free cash can also be used to make acquisitions, or build the cash on the balance sheet.

For more on FCF, read Free Cash Flow and Shareholder Yield by Bill Priest and Lindsay McClelland, Free Cash Flow by George Christy, or Creative Cash Flow Reporting by Charles Mulford and Eugene Comiskey.

# Imports

In [4]:
import FundamentalAnalysis as fa

# Getting Financial Data

In [2]:
api_key = open('/Users/MichaelWirtz/Desktop/pathfile/fundamental_analysis_api_key.txt').read()

In [3]:
api_key

'410c9480ef0825c9d063beecf1536923'

In [13]:
metrics = fa.key_metrics('AAPL',api_key, 'annual')

In [8]:
fcf_metrics

,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
revenuePerShare,1.582026e+01,1.408528e+01,1.339933e+01,1.098444e+01,9.854053e+00,1.015548e+01,7.509360e+00,6.596481e+00,5.979315e+00,4.182853e+00,...,6.909706e-01,5.970808e-01,5.165452e-01,4.683046e-01,3.944082e-01,3.666389e-01,2.798323e-01,1.805292e-01,1.320100e-01,1.383439e-01
netIncomePerShare,3.308587e+00,2.991446e+00,3.003354e+00,2.316885e+00,2.087758e+00,2.320098e+00,1.623101e+00,1.429488e+00,1.594390e+00,1.001653e+00,...,2.332460e-02,6.511976e-03,3.862974e-02,2.301069e-02,3.370707e-02,3.150153e-02,2.749519e-02,1.472185e-02,1.068851e-02,4.399885e-03
operatingCashFlowPerShare,4.649230e+00,3.756685e+00,3.906565e+00,3.047491e+00,3.026557e+00,3.531203e+00,2.453056e+00,2.071305e+00,1.942930e+00,1.450159e+00,...,5.542016e-02,-4.872754e-02,6.712828e-02,1.402910e-02,6.840761e-02,3.517902e-02,2.020896e-02,1.275441e-02,2.186285e-02,1.911425e-02
freeCashFlowPerShare,4.228014e+00,3.188508e+00,3.234920e+00,2.434380e+00,2.444652e+00,3.032022e+00,2.049931e+00,1.721006e+00,1.583731e+00,1.162206e+00,...,4.341962e-02,-6.467066e-02,5.291545e-02,-2.152613e-03,5.251206e-02,1.859561e-02,1.031070e-02,6.919946e-03,1.721266e-02,1.521927e-02
cashPerShare,2.190856e+00,2.644313e+00,1.307317e+00,9.722091e-01,9.360571e-01,9.177149e-01,5.687222e-01,5.503436e-01,4.105459e-01,3.792618e-01,...,9.049925e-02,5.059880e-02,3.637026e-02,4.483373e-02,2.661084e-02,3.039134e-02,2.557053e-02,3.833107e-02,3.997779e-02,2.430756e-02
bookValuePerShare,3.765477e+00,4.898834e+00,5.405593e+00,6.423269e+00,5.860593e+00,5.186262e+00,4.582437e+00,4.768525e+00,4.516158e+00,2.960483e+00,...,1.792182e-01,1.516467e-01,1.594023e-01,1.311609e-01,1.026824e-01,1.031085e-01,6.894418e-02,5.671642e-02,4.816768e-02,3.967109e-02
tangibleBookValuePerShare,1.866562e+01,1.832656e+01,1.845092e+01,1.760049e+01,1.430617e+01,1.222473e+01,9.164340e+00,7.767259e+00,6.521705e+00,4.325439e+00,...,3.987531e-01,3.870509e-01,3.078717e-01,2.593527e-01,2.111837e-01,1.903969e-01,1.431125e-01,1.002714e-01,8.051083e-02,6.751298e-02
shareholdersEquityPerShare,3.765477e+00,4.898834e+00,5.405593e+00,6.423269e+00,5.860593e+00,5.186262e+00,4.582437e+00,4.768525e+00,4.516158e+00,2.960483e+00,...,1.792182e-01,1.516467e-01,1.594023e-01,1.311609e-01,1.026824e-01,1.031085e-01,6.894418e-02,5.671642e-02,4.816768e-02,3.967109e-02
interestDebtPerShare,6.645240e+00,5.909264e+00,5.496000e+00,5.654472e+00,3.883696e+00,2.718426e+00,1.465721e+00,6.598411e-01,0.000000e+00,0.000000e+00,...,4.781454e-02,6.646707e-02,1.800292e-02,1.105998e-02,8.728357e-03,3.955037e-03,2.165246e-02,0.000000e+00,0.000000e+00,0.000000e+00
marketCap,1.996361e+12,1.150072e+12,1.051729e+12,8.698186e+11,6.436420e+11,6.632544e+11,6.396545e+11,4.903144e+11,5.646301e+11,3.750362e+11,...,5.001711e+09,3.608389e+09,5.971877e+09,6.164639e+09,3.774641e+09,5.822707e+09,5.000875e+09,4.672049e+09,2.122601e+09,1.114069e+09


In [21]:
def average_growth(metric, num_years):
    
    listed_metrics = metrics.loc[metric][0:num_years].to_list()
    annual_growth = []
    for i in range(0,(len(listed_metrics) -1)):
        yoy_g = (listed_metrics[i] / listed_metrics[i+1]) - 1
        annual_growth.append(yoy_g)
    
    total_period_growth_rate = sum(annual_growth) / len(annual_growth)
    
    return total_period_growth_rate  

In [ ]:
fcf_roic = pd.DataFrame(columns=['company',
                                 'fcf_per_share_growht',
                                 'roic_growth'])

In [22]:
companies = ['AAPL','AMZN','MSFT']

In [23]:
for company in companies:
    metrics = fa.key_metrics(company,api_key, 'annual')
    fcf_per_share_growth = average_growth('freeCashFlowPerShare',5)
    roic_growth = average_growth('roic',5)
    print(company, fcf_per_share_growth, roic_growth)
    print('\n') 

AAPL 0.1590787740799669 0.05704910963240192


AMZN 0.4268516292376518 0.2259357078119244


MSFT 0.1753919633453116 0.13413202707502622




In [16]:
average_growth('freeCashFlowPerShare',5)

0.1558345550715372

In [20]:
average_growth('roic',8)

0.01179967775161179

In [14]:
fcf_metrics.loc['freeCashFlowPerShare'][0:5]

2020    4.228014
2019    3.188508
2018    3.234920
2017    2.434380
2016    2.444652
Name: freeCashFlowPerShare, dtype: float64